In [ ]:
!uv pip install datasets pylate

Using Python 3.12.12 environment at: /usr
Resolved 74 packages in 1.26s
Prepared 9 packages in 585ms
Uninstalled 2 packages in 380ms
Installed 10 packages in 64ms
 + fast-plaid==1.2.4.280
 + fastkmeans==0.5.0
 + maturin==1.9.6
 + ninja==1.11.1.4
 + pylate==1.3.4
 - setuptools==75.2.0
 + setuptools==80.9.0
 + sqlitedict==2.1.0
 - transformers==4.57.1
 + transformers==4.56.2
 + ujson==5.10.0
 + voyager==2.1.0


In [ ]:
import argparse
import gzip
import json
import logging
import os
import pickle
import random
import tarfile
from datetime import datetime
from datasets import load_dataset, Dataset
from tqdm import tqdm
import pandas as pd
from torch.utils.data import DataLoader

from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    losses,
)
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction, SequentialEvaluator,TripletEvaluator
from sentence_transformers.training_args import BatchSamplers

from sentence_transformers import InputExample, LoggingHandler, SentenceTransformer, losses, models, util

#### Just some code to print debug information to stdout
logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO, handlers=[LoggingHandler()]
)
#### /print debug information to stdout

In [ ]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `full` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `full`


In [ ]:
!uv pip install beir pylate ranx

Using Python 3.12.12 environment at: /usr
Resolved 108 packages in 954ms
Prepared 15 packages in 1.14s
Installed 15 packages in 7ms
 + beir==2.2.0
 + cbor==1.0.0
 + cbor2==5.7.0
 + fastparquet==2024.11.0
 + ijson==3.4.0.post0
 + inscriptis==2.6.0
 + ir-datasets==0.5.11
 + lz4==4.4.4
 + pytrec-eval-terrier==0.5.10
 + ranx==0.3.21
 + trec-car-tools==2.6
 + unlzw3==0.2.3
 + warc3-wet==0.2.5
 + warc3-wet-clueweb09==0.2.5
 + zlib-state==0.1.10


In [ ]:
"""Evaluation script for the custom dataset."""

from __future__ import annotations

import csv
from pathlib import Path

from pylate import evaluation, indexes, models, retrieve
from datasets import load_dataset # Import load_dataset

# Removed CONFIG_PATH and _load_config function as we will define config directly

def _load_hf_information_retrieval_dataset(
    config: dict[str, str]
) -> tuple[list[dict[str, str]], dict[str, str], dict[str, dict[str, float]]]:
    # This function remains to load data from Hugging Face based on the config dict
    corpus_split = load_dataset(
        path=config["hf_corpus_path"],
        split=config["hf_corpus_split"],
    )
    queries_path = config.get("hf_queries_path") or config["hf_corpus_path"]
    queries_split = load_dataset(
        path=queries_path,
        split=config["hf_queries_split"],
    )
    qrels_split = load_dataset(
        path=config["hf_qrels_path"],
        split=config["hf_qrels_split"],
    )

    corpus_id_field = config.get("corpus_id_field", "_id")
    corpus_text_field = config.get("corpus_text_field", "text")
    query_id_field = config.get("query_id_field", "_id")
    query_text_field = config.get("query_text_field", "text")
    qrels_query_field = config.get("qrels_query_field", "query-id")
    qrels_doc_field = config.get("qrels_doc_field", "corpus-id")
    qrels_score_field = config.get("qrels_score_field", "score")

    documents = [
        {
            "id": str(row[corpus_id_field]),
            "text": row[corpus_text_field],
        }
        for row in corpus_split
    ]
    queries = {
        str(row[query_id_field]): row[query_text_field]
        for row in queries_split
    }
    qrels: dict[str, dict[str, float]] = {}
    for row in qrels_split:
        query_id = str(row[qrels_query_field])
        doc_id = str(row[qrels_doc_field])
        score = row[qrels_score_field]
        try:
            score_value = float(score)
        except (TypeError, ValueError):
            score_value = 1.0
        qrels.setdefault(query_id, {})[doc_id] = score_value

    return documents, queries, qrels


def run(dataset_name: str = "SciFact-TR") -> dict[str, float]:
    # Define the configuration directly for SciFact-TR from Hugging Face
    config = {
        "dataset_name": "SciFact-TR",
        "hf_corpus_path": "AbdulkaderSaoud/scifact-tr",
        "hf_corpus_split": "corpus",
        "hf_queries_path": "AbdulkaderSaoud/scifact-tr",
        "hf_queries_split": "queries",
        "hf_qrels_path": "AbdulkaderSaoud/scifact-tr-qrels",
        "hf_qrels_split": "test",
        "query_length": None, # Or specify if needed
        "corpus_id_field": "_id", # Assuming these are correct based on dataset
        "corpus_text_field": "text",
        "query_id_field": "_id",
        "query_text_field": "text",
        "qrels_query_field": "query-id",
        "qrels_doc_field": "corpus-id",
        "qrels_score_field": "score",
        "dataset_path": None # No local path needed when using HF
    }
    # Removed the call to _load_config(dataset_name)

    model = models.ColBERT(
        model_name_or_path="lightonai/GTE-ModernColBERT-v1",
        document_length=300,
        query_length=(
            int(config["query_length"]) if config["query_length"] else None
        ),
    )
    index = indexes.PLAID(override=True)
    retriever = retrieve.ColBERT(index=index)

    # Load dataset using the Hugging Face configuration
    documents, queries, qrels = _load_hf_information_retrieval_dataset(config)

    documents_embeddings = model.encode(
        sentences=[document["text"] for document in documents],
        batch_size=128,
        is_query=False,
        show_progress_bar=True,
    )

    index.add_documents(
        documents_ids=[document["id"] for document in documents],
        documents_embeddings=documents_embeddings,
    )

    queries_embeddings = model.encode(
        sentences=list(queries.values()),
        batch_size=128,
        is_query=True,
        show_progress_bar=True,
    )

    scores = retriever.retrieve(queries_embeddings=queries_embeddings, k=100)

    results = evaluation.evaluate(
        scores=scores,
    qrels=qrels,
    queries=list(queries.keys()),
    metrics=[f"ndcg@{k}" for k in [1, 3, 5, 10, 100]] # NDCG for different k values
    + [f"hits@{k}" for k in [1, 3, 5, 10, 100]]       # Hits at different k values
    + ["map"]                                         # Mean Average Precision (MAP)
    + ["recall@10", "recall@100"]                     # Recall at k
    + ["precision@10", "precision@100"],              # Precision at k
    )

    return results

if __name__ == "__main__":
    metrics = run()
    print(metrics)

Encoding documents (bs=128):   0%|          | 0/41 [00:00<?, ?it/s]

Encoding queries (bs=128):   0%|          | 0/9 [00:00<?, ?it/s]

{'ndcg@1': np.float64(0.4266666666666667), 'ndcg@3': np.float64(0.4992746036759799), 'ndcg@5': np.float64(0.524789707859561), 'ndcg@10': np.float64(0.5466229695944916), 'ndcg@100': np.float64(0.590772372720054), 'hits@1': np.float64(0.4266666666666667), 'hits@3': np.float64(0.5833333333333334), 'hits@5': np.float64(0.6666666666666666), 'hits@10': np.float64(0.75), 'hits@100': np.float64(1.0), 'map': np.float64(0.5076162338244186), 'recall@10': np.float64(0.682), 'recall@100': np.float64(0.8835555555555555), 'precision@10': np.float64(0.075), 'precision@100': np.float64(0.01)}


In [ ]:
"""Evaluation script for the custom dataset."""

from __future__ import annotations

import csv
from pathlib import Path

from pylate import evaluation, indexes, models, retrieve
from datasets import load_dataset # Import load_dataset

# Removed CONFIG_PATH and _load_config function as we will define config directly

def _load_hf_information_retrieval_dataset(
    config: dict[str, str]
) -> tuple[list[dict[str, str]], dict[str, str], dict[str, dict[str, float]]]:
    # This function remains to load data from Hugging Face based on the config dict
    corpus_split = load_dataset(
        path=config["hf_corpus_path"],
        split=config["hf_corpus_split"],
    )
    queries_path = config.get("hf_queries_path") or config["hf_corpus_path"]
    queries_split = load_dataset(
        path=queries_path,
        split=config["hf_queries_split"],
    )
    qrels_split = load_dataset(
        path=config["hf_qrels_path"],
        split=config["hf_qrels_split"],
    )

    corpus_id_field = config.get("corpus_id_field", "_id")
    corpus_text_field = config.get("corpus_text_field", "text")
    query_id_field = config.get("query_id_field", "_id")
    query_text_field = config.get("query_text_field", "text")
    qrels_query_field = config.get("qrels_query_field", "query-id")
    qrels_doc_field = config.get("qrels_doc_field", "corpus-id")
    qrels_score_field = config.get("qrels_score_field", "score")

    documents = [
        {
            "id": str(row[corpus_id_field]),
            "text": row[corpus_text_field],
        }
        for row in corpus_split
    ]
    queries = {
        str(row[query_id_field]): row[query_text_field]
        for row in queries_split
    }
    qrels: dict[str, dict[str, float]] = {}
    for row in qrels_split:
        query_id = str(row[qrels_query_field])
        doc_id = str(row[qrels_doc_field])
        score = row[qrels_score_field]
        try:
            score_value = float(score)
        except (TypeError, ValueError):
            score_value = 1.0
        qrels.setdefault(query_id, {})[doc_id] = score_value

    return documents, queries, qrels


def run(dataset_name: str = "SciFact-TR") -> dict[str, float]:
    # Define the configuration directly for SciFact-TR from Hugging Face
    config = {
        "dataset_name": "SciFact-TR",
        "hf_corpus_path": "AbdulkaderSaoud/scifact-tr",
        "hf_corpus_split": "corpus",
        "hf_queries_path": "AbdulkaderSaoud/scifact-tr",
        "hf_queries_split": "queries",
        "hf_qrels_path": "AbdulkaderSaoud/scifact-tr-qrels",
        "hf_qrels_split": "test",
        "query_length": None, # Or specify if needed
        "corpus_id_field": "_id", # Assuming these are correct based on dataset
        "corpus_text_field": "text",
        "query_id_field": "_id",
        "query_text_field": "text",
        "qrels_query_field": "query-id",
        "qrels_doc_field": "corpus-id",
        "qrels_score_field": "score",
        "dataset_path": None # No local path needed when using HF
    }
    # Removed the call to _load_config(dataset_name)

    model = models.ColBERT(
        model_name_or_path="ozayezerceli/mmBERT-small-TR",
        document_length=300,
        query_length=(
            int(config["query_length"]) if config["query_length"] else None
        ),
    )
    index = indexes.PLAID(override=True)
    retriever = retrieve.ColBERT(index=index)

    # Load dataset using the Hugging Face configuration
    documents, queries, qrels = _load_hf_information_retrieval_dataset(config)

    documents_embeddings = model.encode(
        sentences=[document["text"] for document in documents],
        batch_size=128,
        is_query=False,
        show_progress_bar=True,
    )

    index.add_documents(
        documents_ids=[document["id"] for document in documents],
        documents_embeddings=documents_embeddings,
    )

    queries_embeddings = model.encode(
        sentences=list(queries.values()),
        batch_size=128,
        is_query=True,
        show_progress_bar=True,
    )

    scores = retriever.retrieve(queries_embeddings=queries_embeddings, k=100)

    results = evaluation.evaluate(
        scores=scores,
    qrels=qrels,
    queries=list(queries.keys()),
    metrics=[f"ndcg@{k}" for k in [1, 3, 5, 10, 100]] # NDCG for different k values
    + [f"hits@{k}" for k in [1, 3, 5, 10, 100]]       # Hits at different k values
    + ["map"]                                         # Mean Average Precision (MAP)
    + ["recall@10", "recall@100"]                     # Recall at k
    + ["precision@10", "precision@100"],              # Precision at k
    )

    return results

if __name__ == "__main__":
    metrics = run()
    print(metrics)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/38.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

Encoding documents (bs=128):   0%|          | 0/41 [00:00<?, ?it/s]

Encoding queries (bs=128):   0%|          | 0/9 [00:00<?, ?it/s]

{'ndcg@1': np.float64(0.15666666666666668), 'ndcg@3': np.float64(0.1956026770192575), 'ndcg@5': np.float64(0.2063304094466022), 'ndcg@10': np.float64(0.22615823591609185), 'ndcg@100': np.float64(0.27506972372282557), 'hits@1': np.float64(0.15666666666666668), 'hits@3': np.float64(0.23666666666666666), 'hits@5': np.float64(0.27), 'hits@10': np.float64(0.34), 'hits@100': np.float64(0.61), 'map': np.float64(0.20682764797492206), 'recall@10': np.float64(0.3063888888888889), 'recall@100': np.float64(0.5450555555555555), 'precision@10': np.float64(0.033999999999999996), 'precision@100': np.float64(0.0061)}


In [ ]:
"""Evaluation script for the custom dataset."""

from __future__ import annotations

import csv
from pathlib import Path

from pylate import evaluation, indexes, models, retrieve
from datasets import load_dataset # Import load_dataset

# Removed CONFIG_PATH and _load_config function as we will define config directly

def _load_hf_information_retrieval_dataset(
    config: dict[str, str]
) -> tuple[list[dict[str, str]], dict[str, str], dict[str, dict[str, float]]]:
    # This function remains to load data from Hugging Face based on the config dict
    corpus_split = load_dataset(
        path=config["hf_corpus_path"],
        split=config["hf_corpus_split"],
    )
    queries_path = config.get("hf_queries_path") or config["hf_corpus_path"]
    queries_split = load_dataset(
        path=queries_path,
        split=config["hf_queries_split"],
    )
    qrels_split = load_dataset(
        path=config["hf_qrels_path"],
        split=config["hf_qrels_split"],
    )

    corpus_id_field = config.get("corpus_id_field", "_id")
    corpus_text_field = config.get("corpus_text_field", "text")
    query_id_field = config.get("query_id_field", "_id")
    query_text_field = config.get("query_text_field", "text")
    qrels_query_field = config.get("qrels_query_field", "query-id")
    qrels_doc_field = config.get("qrels_doc_field", "corpus-id")
    qrels_score_field = config.get("qrels_score_field", "score")

    documents = [
        {
            "id": str(row[corpus_id_field]),
            "text": row[corpus_text_field],
        }
        for row in corpus_split
    ]
    queries = {
        str(row[query_id_field]): row[query_text_field]
        for row in queries_split
    }
    qrels: dict[str, dict[str, float]] = {}
    for row in qrels_split:
        query_id = str(row[qrels_query_field])
        doc_id = str(row[qrels_doc_field])
        score = row[qrels_score_field]
        try:
            score_value = float(score)
        except (TypeError, ValueError):
            score_value = 1.0
        qrels.setdefault(query_id, {})[doc_id] = score_value

    return documents, queries, qrels


def run(dataset_name: str = "SciFact-TR") -> dict[str, float]:
    # Define the configuration directly for SciFact-TR from Hugging Face
    config = {
        "dataset_name": "SciFact-TR",
        "hf_corpus_path": "AbdulkaderSaoud/scifact-tr",
        "hf_corpus_split": "corpus",
        "hf_queries_path": "AbdulkaderSaoud/scifact-tr",
        "hf_queries_split": "queries",
        "hf_qrels_path": "AbdulkaderSaoud/scifact-tr-qrels",
        "hf_qrels_split": "test",
        "query_length": None, # Or specify if needed
        "corpus_id_field": "_id", # Assuming these are correct based on dataset
        "corpus_text_field": "text",
        "query_id_field": "_id",
        "query_text_field": "text",
        "qrels_query_field": "query-id",
        "qrels_doc_field": "corpus-id",
        "qrels_score_field": "score",
        "dataset_path": None # No local path needed when using HF
    }
    # Removed the call to _load_config(dataset_name)

    model = models.ColBERT(
        model_name_or_path="ozayezerceli/mmBERT-base-TR",
        document_length=300,
        query_length=(
            int(config["query_length"]) if config["query_length"] else None
        ),
    )
    index = indexes.PLAID(override=True)
    retriever = retrieve.ColBERT(index=index)

    # Load dataset using the Hugging Face configuration
    documents, queries, qrels = _load_hf_information_retrieval_dataset(config)

    documents_embeddings = model.encode(
        sentences=[document["text"] for document in documents],
        batch_size=128,
        is_query=False,
        show_progress_bar=True,
    )

    index.add_documents(
        documents_ids=[document["id"] for document in documents],
        documents_embeddings=documents_embeddings,
    )

    queries_embeddings = model.encode(
        sentences=list(queries.values()),
        batch_size=128,
        is_query=True,
        show_progress_bar=True,
    )

    scores = retriever.retrieve(queries_embeddings=queries_embeddings, k=100)

    results = evaluation.evaluate(
        scores=scores,
    qrels=qrels,
    queries=list(queries.keys()),
    metrics=[f"ndcg@{k}" for k in [1, 3, 5, 10, 100]] # NDCG for different k values
    + [f"hits@{k}" for k in [1, 3, 5, 10, 100]]       # Hits at different k values
    + ["map"]                                         # Mean Average Precision (MAP)
    + ["recall@10", "recall@100"]                     # Recall at k
    + ["precision@10", "precision@100"],              # Precision at k
    )

    return results

if __name__ == "__main__":
    metrics = run()
    print(metrics)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/36.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

Encoding documents (bs=128):   0%|          | 0/41 [00:00<?, ?it/s]

Encoding queries (bs=128):   0%|          | 0/9 [00:00<?, ?it/s]

{'ndcg@1': np.float64(0.06), 'ndcg@3': np.float64(0.06983476786335058), 'ndcg@5': np.float64(0.07732710475346578), 'ndcg@10': np.float64(0.08368388701808091), 'ndcg@100': np.float64(0.11742262436412067), 'hits@1': np.float64(0.06), 'hits@3': np.float64(0.08666666666666667), 'hits@5': np.float64(0.11), 'hits@10': np.float64(0.13333333333333333), 'hits@100': np.float64(0.32), 'map': np.float64(0.07752006997007972), 'recall@10': np.float64(0.11444444444444443), 'recall@100': np.float64(0.2752222222222222), 'precision@10': np.float64(0.013333333333333334), 'precision@100': np.float64(0.0031999999999999997)}


In [ ]:
"""Evaluation script for the custom dataset."""

from __future__ import annotations

import csv
from pathlib import Path

from pylate import evaluation, indexes, models, retrieve
from datasets import load_dataset # Import load_dataset

# Removed CONFIG_PATH and _load_config function as we will define config directly

def _load_hf_information_retrieval_dataset(
    config: dict[str, str]
) -> tuple[list[dict[str, str]], dict[str, str], dict[str, dict[str, float]]]:
    # This function remains to load data from Hugging Face based on the config dict
    corpus_split = load_dataset(
        path=config["hf_corpus_path"],
        split=config["hf_corpus_split"],
    )
    queries_path = config.get("hf_queries_path") or config["hf_corpus_path"]
    queries_split = load_dataset(
        path=queries_path,
        split=config["hf_queries_split"],
    )
    qrels_split = load_dataset(
        path=config["hf_qrels_path"],
        split=config["hf_qrels_split"],
    )

    corpus_id_field = config.get("corpus_id_field", "_id")
    corpus_text_field = config.get("corpus_text_field", "text")
    query_id_field = config.get("query_id_field", "_id")
    query_text_field = config.get("query_text_field", "text")
    qrels_query_field = config.get("qrels_query_field", "query-id")
    qrels_doc_field = config.get("qrels_doc_field", "corpus-id")
    qrels_score_field = config.get("qrels_score_field", "score")

    documents = [
        {
            "id": str(row[corpus_id_field]),
            "text": row[corpus_text_field],
        }
        for row in corpus_split
    ]
    queries = {
        str(row[query_id_field]): row[query_text_field]
        for row in queries_split
    }
    qrels: dict[str, dict[str, float]] = {}
    for row in qrels_split:
        query_id = str(row[qrels_query_field])
        doc_id = str(row[qrels_doc_field])
        score = row[qrels_score_field]
        try:
            score_value = float(score)
        except (TypeError, ValueError):
            score_value = 1.0
        qrels.setdefault(query_id, {})[doc_id] = score_value

    return documents, queries, qrels


def run(dataset_name: str = "SciFact-TR") -> dict[str, float]:
    # Define the configuration directly for SciFact-TR from Hugging Face
    config = {
        "dataset_name": "SciFact-TR",
        "hf_corpus_path": "AbdulkaderSaoud/scifact-tr",
        "hf_corpus_split": "corpus",
        "hf_queries_path": "AbdulkaderSaoud/scifact-tr",
        "hf_queries_split": "queries",
        "hf_qrels_path": "AbdulkaderSaoud/scifact-tr-qrels",
        "hf_qrels_split": "test",
        "query_length": None, # Or specify if needed
        "corpus_id_field": "_id", # Assuming these are correct based on dataset
        "corpus_text_field": "text",
        "query_id_field": "_id",
        "query_text_field": "text",
        "qrels_query_field": "query-id",
        "qrels_doc_field": "corpus-id",
        "qrels_score_field": "score",
        "dataset_path": None # No local path needed when using HF
    }
    # Removed the call to _load_config(dataset_name)

    model = models.ColBERT(
        model_name_or_path="ozayezerceli/ColmmBERT-small-TR",
        document_length=300,
        query_length=(
            int(config["query_length"]) if config["query_length"] else None
        ),
    )
    index = indexes.PLAID(override=True)
    retriever = retrieve.ColBERT(index=index)

    # Load dataset using the Hugging Face configuration
    documents, queries, qrels = _load_hf_information_retrieval_dataset(config)

    documents_embeddings = model.encode(
        sentences=[document["text"] for document in documents],
        batch_size=128,
        is_query=False,
        show_progress_bar=True,
    )

    index.add_documents(
        documents_ids=[document["id"] for document in documents],
        documents_embeddings=documents_embeddings,
    )

    queries_embeddings = model.encode(
        sentences=list(queries.values()),
        batch_size=128,
        is_query=True,
        show_progress_bar=True,
    )

    scores = retriever.retrieve(queries_embeddings=queries_embeddings, k=100)

    results = evaluation.evaluate(
        scores=scores,
    qrels=qrels,
    queries=list(queries.keys()),
    metrics=[f"ndcg@{k}" for k in [1, 3, 5, 10, 100]] # NDCG for different k values
    + [f"hits@{k}" for k in [1, 3, 5, 10, 100]]       # Hits at different k values
    + ["map"]                                         # Mean Average Precision (MAP)
    + ["recall@10", "recall@100"]                     # Recall at k
    + ["precision@10", "precision@100"],              # Precision at k
    )

    return results

if __name__ == "__main__":
    metrics = run()
    print(metrics)

modules.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/771 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/28.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

1_Dense/model.safetensors:   0%|          | 0.00/197k [00:00<?, ?B/s]

Encoding documents (bs=128):   0%|          | 0/41 [00:00<?, ?it/s]

Encoding queries (bs=128):   0%|          | 0/9 [00:00<?, ?it/s]

{'ndcg@1': np.float64(0.47333333333333333), 'ndcg@3': np.float64(0.5467883209069747), 'ndcg@5': np.float64(0.5643680766561286), 'ndcg@10': np.float64(0.5888613239104448), 'ndcg@100': np.float64(0.6205966396570188), 'hits@1': np.float64(0.47333333333333333), 'hits@3': np.float64(0.6333333333333333), 'hits@5': np.float64(0.6933333333333334), 'hits@10': np.float64(0.79), 'hits@100': np.float64(0.96), 'map': np.float64(0.5540773546667423), 'recall@10': np.float64(0.7033888888888888), 'recall@100': np.float64(0.8472222222222223), 'precision@10': np.float64(0.07899999999999999), 'precision@100': np.float64(0.0096)}


In [ ]:
"""Evaluation script for the custom dataset."""

from __future__ import annotations

import csv
from pathlib import Path

from pylate import evaluation, indexes, models, retrieve
from datasets import load_dataset # Import load_dataset

# Removed CONFIG_PATH and _load_config function as we will define config directly

def _load_hf_information_retrieval_dataset(
    config: dict[str, str]
) -> tuple[list[dict[str, str]], dict[str, str], dict[str, dict[str, float]]]:
    # This function remains to load data from Hugging Face based on the config dict
    corpus_split = load_dataset(
        path=config["hf_corpus_path"],
        split=config["hf_corpus_split"],
    )
    queries_path = config.get("hf_queries_path") or config["hf_corpus_path"]
    queries_split = load_dataset(
        path=queries_path,
        split=config["hf_queries_split"],
    )
    qrels_split = load_dataset(
        path=config["hf_qrels_path"],
        split=config["hf_qrels_split"],
    )

    corpus_id_field = config.get("corpus_id_field", "_id")
    corpus_text_field = config.get("corpus_text_field", "text")
    query_id_field = config.get("query_id_field", "_id")
    query_text_field = config.get("query_text_field", "text")
    qrels_query_field = config.get("qrels_query_field", "query-id")
    qrels_doc_field = config.get("qrels_doc_field", "corpus-id")
    qrels_score_field = config.get("qrels_score_field", "score")

    documents = [
        {
            "id": str(row[corpus_id_field]),
            "text": row[corpus_text_field],
        }
        for row in corpus_split
    ]
    queries = {
        str(row[query_id_field]): row[query_text_field]
        for row in queries_split
    }
    qrels: dict[str, dict[str, float]] = {}
    for row in qrels_split:
        query_id = str(row[qrels_query_field])
        doc_id = str(row[qrels_doc_field])
        score = row[qrels_score_field]
        try:
            score_value = float(score)
        except (TypeError, ValueError):
            score_value = 1.0
        qrels.setdefault(query_id, {})[doc_id] = score_value

    return documents, queries, qrels


def run(dataset_name: str = "SciFact-TR") -> dict[str, float]:
    # Define the configuration directly for SciFact-TR from Hugging Face
    config = {
        "dataset_name": "SciFact-TR",
        "hf_corpus_path": "AbdulkaderSaoud/scifact-tr",
        "hf_corpus_split": "corpus",
        "hf_queries_path": "AbdulkaderSaoud/scifact-tr",
        "hf_queries_split": "queries",
        "hf_qrels_path": "AbdulkaderSaoud/scifact-tr-qrels",
        "hf_qrels_split": "test",
        "query_length": None, # Or specify if needed
        "corpus_id_field": "_id", # Assuming these are correct based on dataset
        "corpus_text_field": "text",
        "query_id_field": "_id",
        "query_text_field": "text",
        "qrels_query_field": "query-id",
        "qrels_doc_field": "corpus-id",
        "qrels_score_field": "score",
        "dataset_path": None # No local path needed when using HF
    }
    # Removed the call to _load_config(dataset_name)

    model = models.ColBERT(
        model_name_or_path="ozayezerceli/ColmmBERT-base-TR",
        document_length=300,
        query_length=(
            int(config["query_length"]) if config["query_length"] else None
        ),
    )
    index = indexes.PLAID(override=True)
    retriever = retrieve.ColBERT(index=index)

    # Load dataset using the Hugging Face configuration
    documents, queries, qrels = _load_hf_information_retrieval_dataset(config)

    documents_embeddings = model.encode(
        sentences=[document["text"] for document in documents],
        batch_size=128,
        is_query=False,
        show_progress_bar=True,
    )

    index.add_documents(
        documents_ids=[document["id"] for document in documents],
        documents_embeddings=documents_embeddings,
    )

    queries_embeddings = model.encode(
        sentences=list(queries.values()),
        batch_size=128,
        is_query=True,
        show_progress_bar=True,
    )

    scores = retriever.retrieve(queries_embeddings=queries_embeddings, k=100)

    results = evaluation.evaluate(
        scores=scores,
    qrels=qrels,
    queries=list(queries.keys()),
    metrics=[f"ndcg@{k}" for k in [1, 3, 5, 10, 100]] # NDCG for different k values
    + [f"hits@{k}" for k in [1, 3, 5, 10, 100]]       # Hits at different k values
    + ["map"]                                         # Mean Average Precision (MAP)
    + ["recall@10", "recall@100"]                     # Recall at k
    + ["precision@10", "precision@100"],              # Precision at k
    )

    return results

if __name__ == "__main__":
    metrics = run()
    print(metrics)

modules.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/771 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/24.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

1_Dense/model.safetensors:   0%|          | 0.00/393k [00:00<?, ?B/s]

Encoding documents (bs=128):   0%|          | 0/41 [00:00<?, ?it/s]

Encoding queries (bs=128):   0%|          | 0/9 [00:00<?, ?it/s]

{'ndcg@1': np.float64(0.51), 'ndcg@3': np.float64(0.5561567872163216), 'ndcg@5': np.float64(0.5760758300165963), 'ndcg@10': np.float64(0.5978404295357774), 'ndcg@100': np.float64(0.6319329567624973), 'hits@1': np.float64(0.51), 'hits@3': np.float64(0.63), 'hits@5': np.float64(0.7), 'hits@10': np.float64(0.78), 'hits@100': np.float64(0.97), 'map': np.float64(0.5674619649519079), 'recall@10': np.float64(0.6999444444444445), 'recall@100': np.float64(0.8535555555555555), 'precision@10': np.float64(0.078), 'precision@100': np.float64(0.0097)}


In [ ]:
"""Evaluation script for the custom dataset."""

from __future__ import annotations

import csv
from pathlib import Path

from pylate import evaluation, indexes, models, retrieve
from datasets import load_dataset # Import load_dataset

# Removed CONFIG_PATH and _load_config function as we will define config directly

def _load_hf_information_retrieval_dataset(
    config: dict[str, str]
) -> tuple[list[dict[str, str]], dict[str, str], dict[str, dict[str, float]]]:
    # This function remains to load data from Hugging Face based on the config dict
    corpus_split = load_dataset(
        path=config["hf_corpus_path"],
        split=config["hf_corpus_split"],
    )
    queries_path = config.get("hf_queries_path") or config["hf_corpus_path"]
    queries_split = load_dataset(
        path=queries_path,
        split=config["hf_queries_split"],
    )
    qrels_split = load_dataset(
        path=config["hf_qrels_path"],
        split=config["hf_qrels_split"],
    )

    corpus_id_field = config.get("corpus_id_field", "_id")
    corpus_text_field = config.get("corpus_text_field", "text")
    query_id_field = config.get("query_id_field", "_id")
    query_text_field = config.get("query_text_field", "text")
    qrels_query_field = config.get("qrels_query_field", "query-id")
    qrels_doc_field = config.get("qrels_doc_field", "corpus-id")
    qrels_score_field = config.get("qrels_score_field", "score")

    documents = [
        {
            "id": str(row[corpus_id_field]),
            "text": row[corpus_text_field],
        }
        for row in corpus_split
    ]
    queries = {
        str(row[query_id_field]): row[query_text_field]
        for row in queries_split
    }
    qrels: dict[str, dict[str, float]] = {}
    for row in qrels_split:
        query_id = str(row[qrels_query_field])
        doc_id = str(row[qrels_doc_field])
        score = row[qrels_score_field]
        try:
            score_value = float(score)
        except (TypeError, ValueError):
            score_value = 1.0
        qrels.setdefault(query_id, {})[doc_id] = score_value

    return documents, queries, qrels


def run(dataset_name: str = "SciFact-TR") -> dict[str, float]:
    # Define the configuration directly for SciFact-TR from Hugging Face
    config = {
        "dataset_name": "SciFact-TR",
        "hf_corpus_path": "AbdulkaderSaoud/scifact-tr",
        "hf_corpus_split": "corpus",
        "hf_queries_path": "AbdulkaderSaoud/scifact-tr",
        "hf_queries_split": "queries",
        "hf_qrels_path": "AbdulkaderSaoud/scifact-tr-qrels",
        "hf_qrels_split": "test",
        "query_length": None, # Or specify if needed
        "corpus_id_field": "_id", # Assuming these are correct based on dataset
        "corpus_text_field": "text",
        "query_id_field": "_id",
        "query_text_field": "text",
        "qrels_query_field": "query-id",
        "qrels_doc_field": "corpus-id",
        "qrels_score_field": "score",
        "dataset_path": None # No local path needed when using HF
    }
    # Removed the call to _load_config(dataset_name)

    model = models.ColBERT(
        model_name_or_path="ozayezerceli/ettin-encoder-32M-TR",
        document_length=300,
        query_length=(
            int(config["query_length"]) if config["query_length"] else None
        ),
        trust_remote_code=True,
    )
    index = indexes.PLAID(override=True)
    retriever = retrieve.ColBERT(index=index)

    # Load dataset using the Hugging Face configuration
    documents, queries, qrels = _load_hf_information_retrieval_dataset(config)

    documents_embeddings = model.encode(
        sentences=[document["text"] for document in documents],
        batch_size=128,
        is_query=False,
        show_progress_bar=True,
    )

    index.add_documents(
        documents_ids=[document["id"] for document in documents],
        documents_embeddings=documents_embeddings,
    )

    queries_embeddings = model.encode(
        sentences=list(queries.values()),
        batch_size=128,
        is_query=True,
        show_progress_bar=True,
    )

    scores = retriever.retrieve(queries_embeddings=queries_embeddings, k=100)

    results = evaluation.evaluate(
        scores=scores,
    qrels=qrels,
    queries=list(queries.keys()),
    metrics=[f"ndcg@{k}" for k in [1, 3, 5, 10, 100]] # NDCG for different k values
    + [f"hits@{k}" for k in [1, 3, 5, 10, 100]]       # Hits at different k values
    + ["map"]                                         # Mean Average Precision (MAP)
    + ["recall@10", "recall@100"]                     # Recall at k
    + ["precision@10", "precision@100"],              # Precision at k
    )

    return results

if __name__ == "__main__":
    metrics = run()
    print(metrics)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/35.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Encoding documents (bs=128):   0%|          | 0/41 [00:00<?, ?it/s]

Encoding queries (bs=128):   0%|          | 0/9 [00:00<?, ?it/s]

{'ndcg@1': np.float64(0.09), 'ndcg@3': np.float64(0.11928522272436164), 'ndcg@5': np.float64(0.12635931118094632), 'ndcg@10': np.float64(0.14109262875731157), 'ndcg@100': np.float64(0.17834184913387188), 'hits@1': np.float64(0.09), 'hits@3': np.float64(0.16), 'hits@5': np.float64(0.18), 'hits@10': np.float64(0.24), 'hits@100': np.float64(0.45666666666666667), 'map': np.float64(0.1267664730285408), 'recall@10': np.float64(0.19777777777777777), 'recall@100': np.float64(0.38422222222222224), 'precision@10': np.float64(0.024), 'precision@100': np.float64(0.004566666666666667)}


In [ ]:
"""Evaluation script for the custom dataset."""

from __future__ import annotations

import csv
from pathlib import Path

from pylate import evaluation, indexes, models, retrieve
from datasets import load_dataset # Import load_dataset

# Removed CONFIG_PATH and _load_config function as we will define config directly

def _load_hf_information_retrieval_dataset(
    config: dict[str, str]
) -> tuple[list[dict[str, str]], dict[str, str], dict[str, dict[str, float]]]:
    # This function remains to load data from Hugging Face based on the config dict
    corpus_split = load_dataset(
        path=config["hf_corpus_path"],
        split=config["hf_corpus_split"],
    )
    queries_path = config.get("hf_queries_path") or config["hf_corpus_path"]
    queries_split = load_dataset(
        path=queries_path,
        split=config["hf_queries_split"],
    )
    qrels_split = load_dataset(
        path=config["hf_qrels_path"],
        split=config["hf_qrels_split"],
    )

    corpus_id_field = config.get("corpus_id_field", "_id")
    corpus_text_field = config.get("corpus_text_field", "text")
    query_id_field = config.get("query_id_field", "_id")
    query_text_field = config.get("query_text_field", "text")
    qrels_query_field = config.get("qrels_query_field", "query-id")
    qrels_doc_field = config.get("qrels_doc_field", "corpus-id")
    qrels_score_field = config.get("qrels_score_field", "score")

    documents = [
        {
            "id": str(row[corpus_id_field]),
            "text": row[corpus_text_field],
        }
        for row in corpus_split
    ]
    queries = {
        str(row[query_id_field]): row[query_text_field]
        for row in queries_split
    }
    qrels: dict[str, dict[str, float]] = {}
    for row in qrels_split:
        query_id = str(row[qrels_query_field])
        doc_id = str(row[qrels_doc_field])
        score = row[qrels_score_field]
        try:
            score_value = float(score)
        except (TypeError, ValueError):
            score_value = 1.0
        qrels.setdefault(query_id, {})[doc_id] = score_value

    return documents, queries, qrels


def run(dataset_name: str = "SciFact-TR") -> dict[str, float]:
    # Define the configuration directly for SciFact-TR from Hugging Face
    config = {
        "dataset_name": "SciFact-TR",
        "hf_corpus_path": "AbdulkaderSaoud/scifact-tr",
        "hf_corpus_split": "corpus",
        "hf_queries_path": "AbdulkaderSaoud/scifact-tr",
        "hf_queries_split": "queries",
        "hf_qrels_path": "AbdulkaderSaoud/scifact-tr-qrels",
        "hf_qrels_split": "test",
        "query_length": None, # Or specify if needed
        "corpus_id_field": "_id", # Assuming these are correct based on dataset
        "corpus_text_field": "text",
        "query_id_field": "_id",
        "query_text_field": "text",
        "qrels_query_field": "query-id",
        "qrels_doc_field": "corpus-id",
        "qrels_score_field": "score",
        "dataset_path": None # No local path needed when using HF
    }
    # Removed the call to _load_config(dataset_name)

    model = models.ColBERT(
        model_name_or_path="ozayezerceli/ettin-encoder-150M-TR",
        document_length=300,
        query_length=(
            int(config["query_length"]) if config["query_length"] else None
        ),
        trust_remote_code=True,
    )
    index = indexes.PLAID(override=True)
    retriever = retrieve.ColBERT(index=index)

    # Load dataset using the Hugging Face configuration
    documents, queries, qrels = _load_hf_information_retrieval_dataset(config)

    documents_embeddings = model.encode(
        sentences=[document["text"] for document in documents],
        batch_size=128,
        is_query=False,
        show_progress_bar=True,
    )

    index.add_documents(
        documents_ids=[document["id"] for document in documents],
        documents_embeddings=documents_embeddings,
    )

    queries_embeddings = model.encode(
        sentences=list(queries.values()),
        batch_size=128,
        is_query=True,
        show_progress_bar=True,
    )

    scores = retriever.retrieve(queries_embeddings=queries_embeddings, k=100)

    results = evaluation.evaluate(
        scores=scores,
    qrels=qrels,
    queries=list(queries.keys()),
    metrics=[f"ndcg@{k}" for k in [1, 3, 5, 10, 100]] # NDCG for different k values
    + [f"hits@{k}" for k in [1, 3, 5, 10, 100]]       # Hits at different k values
    + ["map"]                                         # Mean Average Precision (MAP)
    + ["recall@10", "recall@100"]                     # Recall at k
    + ["precision@10", "precision@100"],              # Precision at k
    )

    return results

if __name__ == "__main__":
    metrics = run()
    print(metrics)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/43.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

Encoding documents (bs=128):   0%|          | 0/41 [00:00<?, ?it/s]

Encoding queries (bs=128):   0%|          | 0/9 [00:00<?, ?it/s]

{'ndcg@1': np.float64(0.006666666666666667), 'ndcg@3': np.float64(0.01375809541241917), 'ndcg@5': np.float64(0.01375809541241917), 'ndcg@10': np.float64(0.014348893035032664), 'ndcg@100': np.float64(0.022493876592401835), 'hits@1': np.float64(0.006666666666666667), 'hits@3': np.float64(0.02), 'hits@5': np.float64(0.02), 'hits@10': np.float64(0.023333333333333334), 'hits@100': np.float64(0.07333333333333333), 'map': np.float64(0.01342191975605649), 'recall@10': np.float64(0.019444444444444445), 'recall@100': np.float64(0.061611111111111116), 'precision@10': np.float64(0.002333333333333333), 'precision@100': np.float64(0.0007333333333333334)}


In [ ]:
"""Evaluation script for the custom dataset."""

from __future__ import annotations

import csv
from pathlib import Path

from pylate import evaluation, indexes, models, retrieve
from datasets import load_dataset # Import load_dataset

# Removed CONFIG_PATH and _load_config function as we will define config directly

def _load_hf_information_retrieval_dataset(
    config: dict[str, str]
) -> tuple[list[dict[str, str]], dict[str, str], dict[str, dict[str, float]]]:
    # This function remains to load data from Hugging Face based on the config dict
    corpus_split = load_dataset(
        path=config["hf_corpus_path"],
        split=config["hf_corpus_split"],
    )
    queries_path = config.get("hf_queries_path") or config["hf_corpus_path"]
    queries_split = load_dataset(
        path=queries_path,
        split=config["hf_queries_split"],
    )
    qrels_split = load_dataset(
        path=config["hf_qrels_path"],
        split=config["hf_qrels_split"],
    )

    corpus_id_field = config.get("corpus_id_field", "_id")
    corpus_text_field = config.get("corpus_text_field", "text")
    query_id_field = config.get("query_id_field", "_id")
    query_text_field = config.get("query_text_field", "text")
    qrels_query_field = config.get("qrels_query_field", "query-id")
    qrels_doc_field = config.get("qrels_doc_field", "corpus-id")
    qrels_score_field = config.get("qrels_score_field", "score")

    documents = [
        {
            "id": str(row[corpus_id_field]),
            "text": row[corpus_text_field],
        }
        for row in corpus_split
    ]
    queries = {
        str(row[query_id_field]): row[query_text_field]
        for row in queries_split
    }
    qrels: dict[str, dict[str, float]] = {}
    for row in qrels_split:
        query_id = str(row[qrels_query_field])
        doc_id = str(row[qrels_doc_field])
        score = row[qrels_score_field]
        try:
            score_value = float(score)
        except (TypeError, ValueError):
            score_value = 1.0
        qrels.setdefault(query_id, {})[doc_id] = score_value

    return documents, queries, qrels


def run(dataset_name: str = "SciFact-TR") -> dict[str, float]:
    # Define the configuration directly for SciFact-TR from Hugging Face
    config = {
        "dataset_name": "SciFact-TR",
        "hf_corpus_path": "AbdulkaderSaoud/scifact-tr",
        "hf_corpus_split": "corpus",
        "hf_queries_path": "AbdulkaderSaoud/scifact-tr",
        "hf_queries_split": "queries",
        "hf_qrels_path": "AbdulkaderSaoud/scifact-tr-qrels",
        "hf_qrels_split": "test",
        "query_length": None, # Or specify if needed
        "corpus_id_field": "_id", # Assuming these are correct based on dataset
        "corpus_text_field": "text",
        "query_id_field": "_id",
        "query_text_field": "text",
        "qrels_query_field": "query-id",
        "qrels_doc_field": "corpus-id",
        "qrels_score_field": "score",
        "dataset_path": None # No local path needed when using HF
    }
    # Removed the call to _load_config(dataset_name)

    model = models.ColBERT(
        model_name_or_path="ozayezerceli/bert-hash-nano-TR",
        document_length=300,
        query_length=(
            int(config["query_length"]) if config["query_length"] else None
        ),
        trust_remote_code=True,
    )
    index = indexes.PLAID(override=True)
    retriever = retrieve.ColBERT(index=index)

    # Load dataset using the Hugging Face configuration
    documents, queries, qrels = _load_hf_information_retrieval_dataset(config)

    documents_embeddings = model.encode(
        sentences=[document["text"] for document in documents],
        batch_size=128,
        is_query=False,
        show_progress_bar=True,
    )

    index.add_documents(
        documents_ids=[document["id"] for document in documents],
        documents_embeddings=documents_embeddings,
    )

    queries_embeddings = model.encode(
        sentences=list(queries.values()),
        batch_size=128,
        is_query=True,
        show_progress_bar=True,
    )

    scores = retriever.retrieve(queries_embeddings=queries_embeddings, k=100)

    results = evaluation.evaluate(
        scores=scores,
    qrels=qrels,
    queries=list(queries.keys()),
    metrics=[f"ndcg@{k}" for k in [1, 3, 5, 10, 100]] # NDCG for different k values
    + [f"hits@{k}" for k in [1, 3, 5, 10, 100]]       # Hits at different k values
    + ["map"]                                         # Mean Average Precision (MAP)
    + ["recall@10", "recall@100"]                     # Recall at k
    + ["precision@10", "precision@100"],              # Precision at k
    )

    return results

if __name__ == "__main__":
    metrics = run()
    print(metrics)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/139k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

Encoding documents (bs=128):   0%|          | 0/41 [00:00<?, ?it/s]

Encoding queries (bs=128):   0%|          | 0/9 [00:00<?, ?it/s]

{'ndcg@1': np.float64(0.25666666666666665), 'ndcg@3': np.float64(0.3069389184263595), 'ndcg@5': np.float64(0.33243941733159393), 'ndcg@10': np.float64(0.36064800876614705), 'ndcg@100': np.float64(0.40757767484846996), 'hits@1': np.float64(0.25666666666666665), 'hits@3': np.float64(0.37666666666666665), 'hits@5': np.float64(0.45), 'hits@10': np.float64(0.55), 'hits@100': np.float64(0.81), 'map': np.float64(0.32384160807472645), 'recall@10': np.float64(0.4860555555555555), 'recall@100': np.float64(0.7196111111111111), 'precision@10': np.float64(0.055), 'precision@100': np.float64(0.008100000000000001)}


In [ ]:
"""Evaluation script for the custom dataset."""

from __future__ import annotations

import csv
from pathlib import Path

from pylate import evaluation, indexes, models, retrieve
from datasets import load_dataset # Import load_dataset

# Removed CONFIG_PATH and _load_config function as we will define config directly

def _load_hf_information_retrieval_dataset(
    config: dict[str, str]
) -> tuple[list[dict[str, str]], dict[str, str], dict[str, dict[str, float]]]:
    # This function remains to load data from Hugging Face based on the config dict
    corpus_split = load_dataset(
        path=config["hf_corpus_path"],
        split=config["hf_corpus_split"],
    )
    queries_path = config.get("hf_queries_path") or config["hf_corpus_path"]
    queries_split = load_dataset(
        path=queries_path,
        split=config["hf_queries_split"],
    )
    qrels_split = load_dataset(
        path=config["hf_qrels_path"],
        split=config["hf_qrels_split"],
    )

    corpus_id_field = config.get("corpus_id_field", "_id")
    corpus_text_field = config.get("corpus_text_field", "text")
    query_id_field = config.get("query_id_field", "_id")
    query_text_field = config.get("query_text_field", "text")
    qrels_query_field = config.get("qrels_query_field", "query-id")
    qrels_doc_field = config.get("qrels_doc_field", "corpus-id")
    qrels_score_field = config.get("qrels_score_field", "score")

    documents = [
        {
            "id": str(row[corpus_id_field]),
            "text": row[corpus_text_field],
        }
        for row in corpus_split
    ]
    queries = {
        str(row[query_id_field]): row[query_text_field]
        for row in queries_split
    }
    qrels: dict[str, dict[str, float]] = {}
    for row in qrels_split:
        query_id = str(row[qrels_query_field])
        doc_id = str(row[qrels_doc_field])
        score = row[qrels_score_field]
        try:
            score_value = float(score)
        except (TypeError, ValueError):
            score_value = 1.0
        qrels.setdefault(query_id, {})[doc_id] = score_value

    return documents, queries, qrels


def run(dataset_name: str = "SciFact-TR") -> dict[str, float]:
    # Define the configuration directly for SciFact-TR from Hugging Face
    config = {
        "dataset_name": "SciFact-TR",
        "hf_corpus_path": "AbdulkaderSaoud/scifact-tr",
        "hf_corpus_split": "corpus",
        "hf_queries_path": "AbdulkaderSaoud/scifact-tr",
        "hf_queries_split": "queries",
        "hf_qrels_path": "AbdulkaderSaoud/scifact-tr-qrels",
        "hf_qrels_split": "test",
        "query_length": None, # Or specify if needed
        "corpus_id_field": "_id", # Assuming these are correct based on dataset
        "corpus_text_field": "text",
        "query_id_field": "_id",
        "query_text_field": "text",
        "qrels_query_field": "query-id",
        "qrels_doc_field": "corpus-id",
        "qrels_score_field": "score",
        "dataset_path": None # No local path needed when using HF
    }
    # Removed the call to _load_config(dataset_name)

    model = models.ColBERT(
        model_name_or_path="ozayezerceli/bert-hash-pico-TR",
        document_length=300,
        query_length=(
            int(config["query_length"]) if config["query_length"] else None
        ),
        trust_remote_code=True,
    )
    index = indexes.PLAID(override=True)
    retriever = retrieve.ColBERT(index=index)

    # Load dataset using the Hugging Face configuration
    documents, queries, qrels = _load_hf_information_retrieval_dataset(config)

    documents_embeddings = model.encode(
        sentences=[document["text"] for document in documents],
        batch_size=128,
        is_query=False,
        show_progress_bar=True,
    )

    index.add_documents(
        documents_ids=[document["id"] for document in documents],
        documents_embeddings=documents_embeddings,
    )

    queries_embeddings = model.encode(
        sentences=list(queries.values()),
        batch_size=128,
        is_query=True,
        show_progress_bar=True,
    )

    scores = retriever.retrieve(queries_embeddings=queries_embeddings, k=100)

    results = evaluation.evaluate(
        scores=scores,
    qrels=qrels,
    queries=list(queries.keys()),
    metrics=[f"ndcg@{k}" for k in [1, 3, 5, 10, 100]] # NDCG for different k values
    + [f"hits@{k}" for k in [1, 3, 5, 10, 100]]       # Hits at different k values
    + ["map"]                                         # Mean Average Precision (MAP)
    + ["recall@10", "recall@100"]                     # Recall at k
    + ["precision@10", "precision@100"],              # Precision at k
    )

    return results

if __name__ == "__main__":
    metrics = run()
    print(metrics)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/903 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/311 [00:00<?, ?B/s]

Encoding documents (bs=128):   0%|          | 0/41 [00:00<?, ?it/s]

Encoding queries (bs=128):   0%|          | 0/9 [00:00<?, ?it/s]

{'ndcg@1': np.float64(0.24), 'ndcg@3': np.float64(0.27855337625134924), 'ndcg@5': np.float64(0.29167439455180927), 'ndcg@10': np.float64(0.3132311250085787), 'ndcg@100': np.float64(0.3621502668038531), 'hits@1': np.float64(0.24), 'hits@3': np.float64(0.34), 'hits@5': np.float64(0.38), 'hits@10': np.float64(0.45666666666666667), 'hits@100': np.float64(0.7233333333333334), 'map': np.float64(0.2888630070607402), 'recall@10': np.float64(0.4073333333333333), 'recall@100': np.float64(0.6373888888888889), 'precision@10': np.float64(0.04566666666666667), 'precision@100': np.float64(0.007233333333333333)}


In [ ]:
"""Evaluation script for the custom dataset."""

from __future__ import annotations

import csv
from pathlib import Path

from pylate import evaluation, indexes, models, retrieve
from datasets import load_dataset # Import load_dataset

# Removed CONFIG_PATH and _load_config function as we will define config directly

def _load_hf_information_retrieval_dataset(
    config: dict[str, str]
) -> tuple[list[dict[str, str]], dict[str, str], dict[str, dict[str, float]]]:
    # This function remains to load data from Hugging Face based on the config dict
    corpus_split = load_dataset(
        path=config["hf_corpus_path"],
        split=config["hf_corpus_split"],
    )
    queries_path = config.get("hf_queries_path") or config["hf_corpus_path"]
    queries_split = load_dataset(
        path=queries_path,
        split=config["hf_queries_split"],
    )
    qrels_split = load_dataset(
        path=config["hf_qrels_path"],
        split=config["hf_qrels_split"],
    )

    corpus_id_field = config.get("corpus_id_field", "_id")
    corpus_text_field = config.get("corpus_text_field", "text")
    query_id_field = config.get("query_id_field", "_id")
    query_text_field = config.get("query_text_field", "text")
    qrels_query_field = config.get("qrels_query_field", "query-id")
    qrels_doc_field = config.get("qrels_doc_field", "corpus-id")
    qrels_score_field = config.get("qrels_score_field", "score")

    documents = [
        {
            "id": str(row[corpus_id_field]),
            "text": row[corpus_text_field],
        }
        for row in corpus_split
    ]
    queries = {
        str(row[query_id_field]): row[query_text_field]
        for row in queries_split
    }
    qrels: dict[str, dict[str, float]] = {}
    for row in qrels_split:
        query_id = str(row[qrels_query_field])
        doc_id = str(row[qrels_doc_field])
        score = row[qrels_score_field]
        try:
            score_value = float(score)
        except (TypeError, ValueError):
            score_value = 1.0
        qrels.setdefault(query_id, {})[doc_id] = score_value

    return documents, queries, qrels


def run(dataset_name: str = "SciFact-TR") -> dict[str, float]:
    # Define the configuration directly for SciFact-TR from Hugging Face
    config = {
        "dataset_name": "SciFact-TR",
        "hf_corpus_path": "AbdulkaderSaoud/scifact-tr",
        "hf_corpus_split": "corpus",
        "hf_queries_path": "AbdulkaderSaoud/scifact-tr",
        "hf_queries_split": "queries",
        "hf_qrels_path": "AbdulkaderSaoud/scifact-tr-qrels",
        "hf_qrels_split": "test",
        "query_length": None, # Or specify if needed
        "corpus_id_field": "_id", # Assuming these are correct based on dataset
        "corpus_text_field": "text",
        "query_id_field": "_id",
        "query_text_field": "text",
        "qrels_query_field": "query-id",
        "qrels_doc_field": "corpus-id",
        "qrels_score_field": "score",
        "dataset_path": None # No local path needed when using HF
    }
    # Removed the call to _load_config(dataset_name)

    model = models.ColBERT(
        model_name_or_path="ozayezerceli/bert-hash-femto-TR",
        document_length=300,
        query_length=(
            int(config["query_length"]) if config["query_length"] else None
        ),
        trust_remote_code=True,
    )
    index = indexes.PLAID(override=True)
    retriever = retrieve.ColBERT(index=index)

    # Load dataset using the Hugging Face configuration
    documents, queries, qrels = _load_hf_information_retrieval_dataset(config)

    documents_embeddings = model.encode(
        sentences=[document["text"] for document in documents],
        batch_size=128,
        is_query=False,
        show_progress_bar=True,
    )

    index.add_documents(
        documents_ids=[document["id"] for document in documents],
        documents_embeddings=documents_embeddings,
    )

    queries_embeddings = model.encode(
        sentences=list(queries.values()),
        batch_size=128,
        is_query=True,
        show_progress_bar=True,
    )

    scores = retriever.retrieve(queries_embeddings=queries_embeddings, k=100)

    results = evaluation.evaluate(
        scores=scores,
    qrels=qrels,
    queries=list(queries.keys()),
    metrics=[f"ndcg@{k}" for k in [1, 3, 5, 10, 100]] # NDCG for different k values
    + [f"hits@{k}" for k in [1, 3, 5, 10, 100]]       # Hits at different k values
    + ["map"]                                         # Mean Average Precision (MAP)
    + ["recall@10", "recall@100"]                     # Recall at k
    + ["precision@10", "precision@100"],              # Precision at k
    )

    return results

if __name__ == "__main__":
    metrics = run()
    print(metrics)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/59.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/903 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/311 [00:00<?, ?B/s]

Encoding documents (bs=128):   0%|          | 0/41 [00:00<?, ?it/s]

Encoding queries (bs=128):   0%|          | 0/9 [00:00<?, ?it/s]

{'ndcg@1': np.float64(0.16333333333333333), 'ndcg@3': np.float64(0.18956713148011037), 'ndcg@5': np.float64(0.19710842664171324), 'ndcg@10': np.float64(0.2094731636030391), 'ndcg@100': np.float64(0.2545512957859878), 'hits@1': np.float64(0.16333333333333333), 'hits@3': np.float64(0.24333333333333335), 'hits@5': np.float64(0.26666666666666666), 'hits@10': np.float64(0.31333333333333335), 'hits@100': np.float64(0.5666666666666667), 'map': np.float64(0.19300441947120084), 'recall@10': np.float64(0.26899999999999996), 'recall@100': np.float64(0.4825555555555556), 'precision@10': np.float64(0.03133333333333333), 'precision@100': np.float64(0.005666666666666667)}


In [ ]:
"""Evaluation script for the custom dataset."""

from __future__ import annotations

import csv
from pathlib import Path

from pylate import evaluation, indexes, models, retrieve
from datasets import load_dataset # Import load_dataset

# Removed CONFIG_PATH and _load_config function as we will define config directly

def _load_hf_information_retrieval_dataset(
    config: dict[str, str]
) -> tuple[list[dict[str, str]], dict[str, str], dict[str, dict[str, float]]]:
    # This function remains to load data from Hugging Face based on the config dict
    corpus_split = load_dataset(
        path=config["hf_corpus_path"],
        split=config["hf_corpus_split"],
    )
    queries_path = config.get("hf_queries_path") or config["hf_corpus_path"]
    queries_split = load_dataset(
        path=queries_path,
        split=config["hf_queries_split"],
    )
    qrels_split = load_dataset(
        path=config["hf_qrels_path"],
        split=config["hf_qrels_split"],
    )

    corpus_id_field = config.get("corpus_id_field", "_id")
    corpus_text_field = config.get("corpus_text_field", "text")
    query_id_field = config.get("query_id_field", "_id")
    query_text_field = config.get("query_text_field", "text")
    qrels_query_field = config.get("qrels_query_field", "query-id")
    qrels_doc_field = config.get("qrels_doc_field", "corpus-id")
    qrels_score_field = config.get("qrels_score_field", "score")

    documents = [
        {
            "id": str(row[corpus_id_field]),
            "text": row[corpus_text_field],
        }
        for row in corpus_split
    ]
    queries = {
        str(row[query_id_field]): row[query_text_field]
        for row in queries_split
    }
    qrels: dict[str, dict[str, float]] = {}
    for row in qrels_split:
        query_id = str(row[qrels_query_field])
        doc_id = str(row[qrels_doc_field])
        score = row[qrels_score_field]
        try:
            score_value = float(score)
        except (TypeError, ValueError):
            score_value = 1.0
        qrels.setdefault(query_id, {})[doc_id] = score_value

    return documents, queries, qrels


def run(dataset_name: str = "SciFact-TR") -> dict[str, float]:
    # Define the configuration directly for SciFact-TR from Hugging Face
    config = {
        "dataset_name": "SciFact-TR",
        "hf_corpus_path": "AbdulkaderSaoud/scifact-tr",
        "hf_corpus_split": "corpus",
        "hf_queries_path": "AbdulkaderSaoud/scifact-tr",
        "hf_queries_split": "queries",
        "hf_qrels_path": "AbdulkaderSaoud/scifact-tr-qrels",
        "hf_qrels_split": "test",
        "query_length": None, # Or specify if needed
        "corpus_id_field": "_id", # Assuming these are correct based on dataset
        "corpus_text_field": "text",
        "query_id_field": "_id",
        "query_text_field": "text",
        "qrels_query_field": "query-id",
        "qrels_doc_field": "corpus-id",
        "qrels_score_field": "score",
        "dataset_path": None # No local path needed when using HF
    }
    # Removed the call to _load_config(dataset_name)

    model = models.ColBERT(
        model_name_or_path="ozayezerceli/col-ettin-encoder-150M-TR",
        document_length=300,
        query_length=(
            int(config["query_length"]) if config["query_length"] else None
        ),
        trust_remote_code=True,
    )
    index = indexes.PLAID(override=True)
    retriever = retrieve.ColBERT(index=index)

    # Load dataset using the Hugging Face configuration
    documents, queries, qrels = _load_hf_information_retrieval_dataset(config)

    documents_embeddings = model.encode(
        sentences=[document["text"] for document in documents],
        batch_size=128,
        is_query=False,
        show_progress_bar=True,
    )

    index.add_documents(
        documents_ids=[document["id"] for document in documents],
        documents_embeddings=documents_embeddings,
    )

    queries_embeddings = model.encode(
        sentences=list(queries.values()),
        batch_size=128,
        is_query=True,
        show_progress_bar=True,
    )

    scores = retriever.retrieve(queries_embeddings=queries_embeddings, k=100)

    results = evaluation.evaluate(
        scores=scores,
    qrels=qrels,
    queries=list(queries.keys()),
    metrics=[f"ndcg@{k}" for k in [1, 3, 5, 10, 100]] # NDCG for different k values
    + [f"hits@{k}" for k in [1, 3, 5, 10, 100]]       # Hits at different k values
    + ["map"]                                         # Mean Average Precision (MAP)
    + ["recall@10", "recall@100"]                     # Recall at k
    + ["precision@10", "precision@100"],              # Precision at k
    )

    return results

if __name__ == "__main__":
    metrics = run()
    print(metrics)

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/596M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/581 [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/339 [00:00<?, ? examples/s]

Encoding documents (bs=128):   0%|          | 0/41 [00:00<?, ?it/s]

Encoding queries (bs=128):   0%|          | 0/9 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/ranx/data_structures/report.py:202: SyntaxWarning: invalid escape sequence '\c'
  + """\\begin{table*}[ht]\n\centering\n\caption{\nOverall effectiveness of the models.\nThe best results are highlighted in boldface.\nSuperscripts denote significant differences in """
/usr/local/lib/python3.12/dist-packages/ranx/data_structures/report.py:204: SyntaxWarning: invalid escape sequence '\l'
  + """ with $p \le """
/usr/local/lib/python3.12/dist-packages/ranx/data_structures/report.py:211: SyntaxWarning: invalid escape sequence '\#'
  + "\n\\textbf{\#}"
/usr/local/lib/python3.12/dist-packages/ranx/data_structures/report.py:216: SyntaxWarning: invalid escape sequence '\m'
  + " \\\\ \n\midrule"
/usr/local/lib/python3.12/dist-packages/ranx/data_structures/report.py:246: SyntaxWarning: invalid escape sequence '\e'
  "\\bottomrule\n\end{tabular}\n}\n\label{tab:results}\n\end{table*}"
/usr/local/lib/python3.12/dist-packages/ranx/metrics/ndcg.py:72: NumbaTypeS

{'ndcg@1': np.float64(0.3233333333333333), 'ndcg@3': np.float64(0.3992827604351613), 'ndcg@5': np.float64(0.4269153453281907), 'ndcg@10': np.float64(0.4441561323512693), 'ndcg@100': np.float64(0.49030205734523574), 'hits@1': np.float64(0.3233333333333333), 'hits@3': np.float64(0.49666666666666665), 'hits@5': np.float64(0.5766666666666667), 'hits@10': np.float64(0.64), 'hits@100': np.float64(0.9033333333333333), 'map': np.float64(0.40499673853647283), 'recall@10': np.float64(0.5774444444444445), 'recall@100': np.float64(0.7922222222222223), 'precision@10': np.float64(0.064), 'precision@100': np.float64(0.009033333333333332)}
